In [1]:
# Keras fix
import tensorflow
from tensorflow.python.ops import control_flow_ops
tensorflow.python.control_flow_ops = control_flow_ops

from keras.models import load_model
from nl.text_gen import generate_from_model
from nl.utils import load_w2v_data, GenerateSamples, print_word_list, parse_text

Using TensorFlow backend.
/home/m20163692/anaconda3/envs/tf/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
print('Loading files...')

text_file = 'text_files/asimov'
model_file = 'models/asimov_model.h5'
mode = 'general_1'

with tensorflow.device('/cpu:0'):
    model = load_model(model_file)

seq_len = model.input_shape[1]
input_dim = model.input_shape[2]

word2idx, idx2word, embeddings = load_w2v_data('w2v_embs/asimov/asimov_vocab.json',
                                               'w2v_embs/asimov/asimov_embeddings.npz')

Loading files...


In [3]:
# Random samples from text_file
n_samples = 3
n_words = 150
print_truth = True
n_truth_words = 30
temperatures = [0.25, 0.5, 0.75]

samples = GenerateSamples(word2idx, text_file, mode, 'dir', n_samples, n_words)

for idx, sample in enumerate(samples):
    print('-' * 80)
    print('SAMPLE %d:' % (idx + 1))
    print_word_list(sample[:seq_len])
    
    if print_truth:
        print('\nTRUTH:')
        print_word_list(sample[:seq_len + n_truth_words])
    
    for temperature in temperatures:
        print('\nGENERATED FROM MODEL (temperature = %f):' % temperature)
        model_sample = generate_from_model(sample, model, embeddings, idx2word,
                                           word2idx, n_words, temperature, mode=mode)
        print_word_list(model_sample)
    
    print('-' * 80)

--------------------------------------------------------------------------------
SAMPLE 1:
— ¡oh, un mensaje! — exclamó anthor —.

TRUTH:
— ¡oh, un mensaje! — exclamó anthor —. ¿y qué decía? 
 — bueno, no estoy seguro; eran sólo cinco breves palabras. pero es interesante. 
 — escuche — interrumpió semic con

GENERATED FROM MODEL (temperature = 0.250000):
— ¡oh, un mensaje! — exclamó anthor —. ¿qué es lo que le ha dicho? 
 — ¿qué es lo que ha hecho, señor? — preguntó. 
 — no, no lo sé. 
 — ¿qué es lo que no está, señor? 
 — no, no. 
 — ¿y si yo lo lo? 
 — no lo sé. 
 — ¿y si no le hace daño? 
 — no, no. pero no es que lo haga. 
 — ¿cómo se puede decir que el doctor amadiro es un robot? 
 — no, no lo sé. 
 — ¿cómo se puede decir que el doctor fastolfe es un robot? 
 — ¿qué es lo que no te parece? 
 — porque no lo sé. 
 — ¿por qué?

GENERATED FROM MODEL (temperature = 0.500000):
— ¡oh, un mensaje! — exclamó anthor —. ¿por qué no, doctor fastolfe? 
 — porque es un mundo donde la tierra es 

In [3]:
# Using custom text
text = 'en el centro de nuestra galaxia comenzó'
n_words = 100
temperatures = [0.25, 0.5, 0.75]

print('SOURCE TEXT:\n%s' % text)

for temperature in temperatures:
    print('\nGENERATED FROM MODEL (temperature = %f):' % temperature)
    model_sample = generate_from_model(text, model, embeddings, idx2word, word2idx,
                                       n_words, temperature, truncating='post',
                                       custom_text=True, mode=mode)
    print_word_list(model_sample)

SOURCE TEXT:
en el centro de nuestra galaxia comenzó

GENERATED FROM MODEL (temperature = 0.250000):
en el centro de nuestra galaxia comenzó a estar de pie, y el otro, a un lado, se había dado cuenta de que el robot no era un robot, y el robot se la por la que se había ido, la más importante, el más pequeño de los mundos y el mismo los que han sido de la tierra, ¿verdad? 
 — sí, señor. 
 — ¿y si no es un robot? 
 — no, no lo sé. 
 — ¿y si no lo sea? 
 — ¿por qué

GENERATED FROM MODEL (temperature = 0.500000):
en el centro de nuestra galaxia comenzó la vez más a la vez. el doctor fastolfe se inclinó hacia atrás, y se fue de lo más. 
 — ¿y qué es lo que nos? 
 — no, no. no me — dijo, y añadió —: ¡no, no, no! en aquel momento... el que había estado en el otro extremo de la galaxia se hizo de nuevo la vida en la que se había dado cuenta, y después de todo, la respuesta de la palabra era la de un robot y un

GENERATED FROM MODEL (temperature = 0.750000):
en el centro de nuestra galaxia come

# Checking words

In [15]:
import nltk

tags_corpus = nltk.corpus.conll2002.tagged_words()
tags_dict = {word:tag for word, tag in tags_corpus[:25000]}

In [87]:
i = 0
www = []

for word, tag in tags_corpus[:25000]:
    word = word.lower()
    if i == 25:
        break
        
    if tag == 'PP':
        try:
            www.index(word)
        except:
            print(word)
            www.append(word)
            i += 1

lo
le
ellos
la
ellas
nos
nosotros
los
sí
él
les
yo
ello
ella


In [118]:
import numpy as np
from nl.text_gen import get_next_word_distribution

text = 'bien, en ese caso, hija mía, el depsec'
temperature = 0.65
n_words = 300

t = parse_text(text, word2idx, mode)
tags = []
tagged_words = []

# def get_tag():
    
for i in range(n_words):
    last_word = t[-1]
    preds = get_next_word_distribution(t, model, embeddings, word2idx, temperature, mode=mode)
    probs = np.random.multinomial(5, preds, 1).reshape(-1)
    words = np.argsort(probs)[::-1]
    test = 0
    next_word = idx2word[words[test]]
    
    if last_word == next_word:
        test += 1
        next_word = words[test]
    
#     if 5-np.max(probs) < 2:
#         print(next_word, '-', 5-np.max(probs))
    
    success = -1
    tries = 0
    
    while(success != 0):
        success = 0
        
        if tries > 0:
            test += 1
#             print('-try-')

        next_word = idx2word[words[test]]
        
        try:
            this_tag = tags_dict[next_word]
        except:
            this_tag = None

        if this_tag:
            try :
                if tagged_words[-1] == tagged_words[-2] == next_word:
                    success -= 1
                    
                if tags[-1] == this_tag == 'DA':
                    success -= 1
                    
                if tags[-1] == this_tag == 'PP':
                    success -= 1
                    
                if tags[-1] == this_tag == 'VSI':
                    success -= 1
            except:
                None
                
        tries += 1
       
    tags.append(this_tag)
    
    if this_tag: 
        tagged_words.append(next_word)
    
    t.append(next_word)
    
print_word_list(t)

bien, en ese caso, hija mía, el depsec es muy así. 
 — ¿de qué está hablando? — preguntó trevize. 
 — si se lo has dicho, pero no me ha de ser un robot. 
 — ¿qué es lo que se ha dicho? — preguntó el doctor, con un gesto de la cabeza. 
 — ¿cómo es posible que te lo sea? 
 — ¿qué es lo que no te parece? 
 — porque yo no he oído hablar de él. 
 — no, no. ¿cómo quiere que lo haga? 
 — le he dicho que el señor de la tierra es un mundo. 
 — ¿qué es lo que le dijo? 
 — no lo sé. 
 — ¿cómo lo sabe? 
 — ¿qué es lo que hizo? 
 — ¿qué es lo que no me sé? — preguntó trevize. 
 — sí, pero ¿cómo puedo decir que la gente de la tierra se está bien, y que no hay nada que se le de no ver a un robot. 
 — ¿por qué no? 
 — porque es un robot. 
 — ¿por qué no? 
 — porque es un otro mundo de la galaxia que no es más que un robot. 
 — ¿qué es lo que no te parece? 
 — es posible que el planeta sea, pero no es muy posible que los espaciales se han dado cuenta de que es usted un robot. 
 — ¿y si las cosas se han